In [71]:
import open3d as o3d
import numpy as np
import math


def default_visualization(object, zoom=1.0):
    azimuth_deg = -45
    elevation_deg = -135

    az = math.radians(azimuth_deg)
    el = math.radians(elevation_deg)

    # spherical → cartesian (camera front vector)
    front = np.array([
    math.cos(el) * math.cos(az),
    math.cos(el) * math.sin(az),
    math.sin(el)])

    # Open3D camera looks toward object
    front = -front
    lookat = [0, 0, 0]
    up = [0, 0, 1]  # typical CAD Z-up

    o3d.visualization.draw_geometries(
        object,
        window_name="Default Visualization",
        width=1024,
        height=768,
        lookat=[0, 0, 0],
        up=[0, 0, 1],
        front=front,
        zoom=zoom
    )

SOURCE_PATH = "workpiece/z_bracket.stl" #  CAD model

visualization = []

mesh = o3d.io.read_triangle_mesh(SOURCE_PATH)
mesh.compute_vertex_normals() # Makes the shading look better
    
pcd_edge = o3d.geometry.PointCloud()
pcd_edge.points = mesh.vertices
pcd_edge.normals = mesh.vertex_normals
pcd_edge.paint_uniform_color([1, 0, 0])
visualization.append(pcd_edge)

pcd_all = o3d.geometry.PointCloud()
pcd_all = mesh.sample_points_poisson_disk(number_of_points=5000)
pcd_all.paint_uniform_color([0.25, 0, 0])
# visualization.append(pcd_all)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50, origin=[0, 0, 0])
visualization.append(world_frame)


default_visualization([mesh]+visualization, zoom=1.0)

In [3]:
# Removes duplicate points and averages the normals

pcd_clean = pcd_edge.voxel_down_sample(voxel_size=0.001) 

print(f"Before removing duplicate: {len(pcd_edge.points)} points")
print(f"After removing duplicate: {len(pcd_clean.points)} points")

default_visualization([pcd_clean, mesh], zoom=1.0)


Before removing duplicate: 3254 points
After removing duplicate: 1092 points


In [5]:
# Reduce points that are too close to each other by using a voxel grid approach

closeness_threshold = 5.0  # Units (e.g., mm)

# This returns a new point cloud where points within 
# the same 2x2x2 cube are combined.
pcd_reduced = pcd_clean.voxel_down_sample(voxel_size=closeness_threshold)

print(f"Before combining: {len(pcd_clean.points)} points")
print(f"After combining: {len(pcd_reduced.points)} points")

default_visualization([pcd_reduced, mesh], zoom=1.0)



Before combining: 1092 points
After combining: 161 points


In [84]:
# Generate 6D viewpoints (X, Y, Z, Roll, Pitch, Yaw) for each point

def get_euler_from_matrix(R):
    """Extracts Roll, Pitch, Yaw from a 3x3 Rotation Matrix"""
    # Using the standard XYZ convention
    pitch = math.asin(-R[2, 0])
    roll = math.atan2(R[2, 1], R[2, 2])
    yaw = math.atan2(R[1, 0], R[0, 0])
    
    # Return as degrees for easier reading
    return [math.degrees(roll), math.degrees(pitch), math.degrees(yaw)]


viewpoints_6d = []
viewpoints_visualization = []
offset_distance = 600.0 # 300 mm from the surface along the normal direction

for i in range(len(pcd_reduced.points)):
    P = np.asarray(pcd_reduced.points[i])
    N = np.asarray(pcd_reduced.normals[i])
    
    # 1. Position the camera at the offset
    cam_pos = P + (N * offset_distance)
    
    # 2. Define the target direction
    # We want the camera's Z-axis to point toward P. 
    # Since the camera is at cam_pos, the direction is (P - cam_pos), 
    # which is exactly the same as -N.
    z_axis = -N / np.linalg.norm(N) # Ensure it's a unit vector
    
    # 3. Find the rotation matrix that aligns the world Z [0,0,1] to our z_axis
    # We use a helper function to create a rotation matrix from a direction
    # Create an arbitrary "up" vector (world Y or X) to build the coordinate system
    up = np.array([0, 1, 0]) if abs(z_axis[1]) < 0.9 else np.array([1, 0, 0])
    x_axis = np.cross(up, z_axis)
    x_axis /= np.linalg.norm(x_axis)
    y_axis = np.cross(z_axis, x_axis)
    
    # Build the 3x3 rotation matrix
    R = np.column_stack((x_axis, y_axis, z_axis))

    # 4. Create and transform the coordinate frame
    cam_frame_visualization = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10)
    
    # Apply rotation first, then translation
    cam_frame_visualization.rotate(R, center=(0, 0, 0)) 
    cam_frame_visualization.translate(cam_pos)
    
    viewpoints_visualization.append(cam_frame_visualization)

    # 1. Get the 3 angles from the matrix
    angles = get_euler_from_matrix(R)
    
    # 2. Combine Position (3D) + Angles (3D) = 6D
    pose_6d = [
        float(cam_pos[0]), float(cam_pos[1]), float(cam_pos[2]), # X, Y, Z
        angles[0],  angles[1],  angles[2]   # Roll, Pitch, Yaw
    ]
    
    viewpoints_6d.append(pose_6d)

print("Generated viewpoints preview (X, Y, Z, Roll, Pitch, Yaw):")
for i in range(5):
    print(viewpoints_6d[i])  # Print the first 5 | x y z roll pitch yaw

default_visualization(viewpoints_visualization + [pcd_reduced, mesh], zoom=0.6)

Generated viewpoints preview (X, Y, Z, Roll, Pitch, Yaw):
[230.5, -329.44271909999156, 189.38543819998318, 132.83208850221317, -48.1896851042214, 180.0]
[-230.5, -240.0, -200.0, -35.264389682754654, 44.99999999999999, 0.0]
[377.4698921289734, 23.529806644182674, 238.0, 179.5553904947609, -63.433223665100925, 180.0]
[-14.029806644182674, -367.9698921289734, 238.0, 116.57022597541055, -0.9940374212612265, 180.0]
[-357.2180803812885, -168.0, -103.24850395332766, -26.652527293823407, 69.9999809360483, 0.0]


In [85]:
# Optional Reduction Viewpoint

z_threshold = 300.0 # mm

# Option 1: Using zip to create both new lists at once
viewpoints_6d_reduced = []
viewpoints_visualization_reduced = []

for vp, vis in zip(viewpoints_6d, viewpoints_visualization):
    if vp[2] >= z_threshold:
        viewpoints_6d_reduced.append(vp)
        viewpoints_visualization_reduced.append(vis)


print("Before reduction: ", len(viewpoints_6d))
print("After reduction: ", len(viewpoints_6d_reduced))

default_visualization(viewpoints_visualization_reduced + [pcd_reduced, mesh], zoom=1.25)


Before reduction:  161
After reduction:  27


In [ ]:
# Check the visualization of one viewpoint in detail, including the camera position, target point, and the line connecting them.

def inspect_viewpoint(target_idx, pcd_reduced, viewpoints_6d, viewpoints_visualization, mesh=None):
    target_point_coords = np.asarray(pcd_reduced.points[target_idx])

    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
    sphere.translate(target_point_coords)
    sphere.paint_uniform_color([1, 0, 0])

    selected_camera = viewpoints_visualization[target_idx]

    cam_pos = np.array(viewpoints_6d[target_idx][:3])

    line_points = [target_point_coords, cam_pos]
    line_indices = [[0, 1]]

    connector_line = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(line_points),
        lines=o3d.utility.Vector2iVector(line_indices)
    )
    connector_line.paint_uniform_color([0, 1, 0])

    print(f"\n--- Inspecting Viewpoint Index: {target_idx} ---")
    print(f"Target Surface Point: {target_point_coords}")
    print(f"Camera Position (6D): {viewpoints_6d[target_idx]}")

    return sphere, selected_camera, connector_line, cam_pos, target_point_coords


def camera_pov_visualization(geoms, cam_pos, target_point, show_world_frame=True, show_camera_frame=True, frame_size=50):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # world frame
    if show_world_frame:
        world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=frame_size
        )
        vis.add_geometry(world_frame)

    for g in geoms:
        vis.add_geometry(g)

    ctr = vis.get_view_control()

    # STEP 1 — camera + target
    cam_pos = np.asarray(cam_pos, dtype=float)
    target_point = np.asarray(target_point, dtype=float)
    # print("cam_pos:", cam_pos)
    # print("target_point:", target_point)

    # STEP 2 — forward (camera → target)  
    forward = target_point - cam_pos
    forward = forward / np.linalg.norm(forward)
    # print("forward:", forward)
    
    # STEP 3 — stable world up
    world_up = np.array([0.0, 0.0, 1.0])
    if abs(np.dot(forward, world_up)) > 0.99:
        world_up = np.array([0.0, 1.0, 0.0])
    # print("world_up:", world_up)
    
    # STEP 4 — right vector
    right = np.cross(forward, world_up)
    right = right / np.linalg.norm(right)
    # print("right:", right)
    
    # STEP 5 — true up
    up = np.cross(right, forward)
    up = up / np.linalg.norm(up)
    # print("up:", up)
    
    # OPTIONAL — visualize camera frame
    if show_camera_frame:
        cam_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=frame_size)
        cam_frame.rotate(np.stack([right, up, forward], axis=1), center=(0, 0, 0))
        cam_frame.translate(cam_pos)
        vis.add_geometry(cam_frame)

    
    # STEP 6 — build rotation (your verified correct version)
    R = np.stack([right, -up, forward], axis=1)
    # print("R:\n", R)
    # print("det(R):", np.linalg.det(R))

    # STEP 7 — extrinsic
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = R.T
    extrinsic[:3, 3] = -R.T @ cam_pos
    # print("extrinsic:\n", extrinsic)

    # STEP 8 — apply camera
    param = ctr.convert_to_pinhole_camera_parameters()
    param.extrinsic = extrinsic
    ctr.convert_from_pinhole_camera_parameters(param)

    vis.run()
    vis.destroy_window()

index = 3
# for index in range(27):
sphere, selected_camera, connector_line, cam_pos, target_point_coords = inspect_viewpoint(
    index, pcd_reduced, viewpoints_6d_reduced, viewpoints_visualization_reduced, mesh)

default_visualization([mesh, selected_camera, sphere, connector_line, world_frame], zoom=1.25)

camera_pov_visualization([mesh, sphere, connector_line], cam_pos, target_point_coords, show_world_frame=False, show_camera_frame=False)



--- Inspecting Viewpoint Index: 3 ---
Target Surface Point: [-17.5  32.   38. ]
Camera Position (6D): [225.11641550749576, -168.0, 352.99976897167465, 153.25979406528558, -34.99998269794345, 180.0]


In [9]:
# Remove hidden points from the camera's perspective using Open3D's Hidden Point Removal (HPR) algorithm

def hidden_point_removal_from_camera(pcd, cam_pos, radius_scale=100):
    """
    Returns a visible-point point cloud using Open3D HPR.
    """

    pts = np.asarray(pcd.points)
    diameter = np.linalg.norm(pcd.get_max_bound() - pcd.get_min_bound())

    radius = diameter * radius_scale

    _, pt_map = pcd.hidden_point_removal(cam_pos, radius)
    visible_pcd = pcd.select_by_index(pt_map)

    visible_pcd.paint_uniform_color([0, 1, 0])  # Visible points in green

    return visible_pcd

visible_pcd = hidden_point_removal_from_camera( pcd_all, cam_pos, radius_scale=400)

camera_pov_visualization([visible_pcd, sphere, connector_line], cam_pos, target_point_coords, show_world_frame=False, show_camera_frame=False)


In [10]:
o3d.visualization.draw_geometries([visible_pcd])